In [1]:
import sys

sys.path.insert(0, '../Lab/')
sys.path.insert(0, '../New_Hybrids/')
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from UserIcmKNNCFRecommender import UserIcmKNNCFRecommender
from ItemIcmKNNCFRecommender import ItemIcmKNNCFRecommender
from WeightedHybridV2 import WeightedHybridScoreRecommender
from optimize_weights import optimize_weights
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from GraphBased.P3alphaRecommender import P3alphaRecommender
from GraphBased.RP3betaRecommender import RP3betaRecommender
from MatrixFactorization.IALSRecommender import IALSRecommender

import matplotlib
import matplotlib.pyplot as plt
from Base.Similarity.Compute_Similarity_Python import Compute_Similarity_Python

%matplotlib inline 

import numpy as np

In [2]:
sys.path.insert(0, '../datasets/')
from load_tables import load_ICM_csr, load_URM_csr
ICM_all = load_ICM_csr()
URM_all = load_URM_csr()

In [3]:
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

In [4]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

In [5]:
def plotta(t,maps):
    fig, ax = plt.subplots()
    ax.plot(t, maps)

    ax.set(xlabel='T', ylabel='MAP')
    ax.grid()
    plt.show()

In [6]:
only_URM = {
    "URM_train" : URM_train
}

cython = {
    "URM_train" : URM_train,
    "recompile_cython" : False,
    "verbose" : False
}

also_ICM = {
    "URM_train" : URM_train,
    "ICM" : ICM_all
}


userknn = {}
userknn["topK"] = 102
userknn["shrink"] = 1

itemknn = {}
itemknn["topK"] = 141
itemknn["shrink"] = 47

cy = {}
cy["epochs"] = 400
cy["topK"] = 900
cy["positive_threshold_BPR"] = 7e-18
cy["learning_rate"] = 0.0002
cy["batch_size"] = 10
cy["sgd_mode"] = "sdg"

uicm = {}
uicm["topK"] = 181
uicm["shrink"] = 0.1
uicm["normalize"] = True

iicm = {}
iicm["topK"] = 893
iicm["shrink"] = 2
iicm["normalize"] = True

alpha = {}
alpha["alpha"] = 0.3784740936494376
alpha["topK"] = 500


beta= {}
beta["topK"]=500
beta["alpha"]=0.3784740936494376
beta["beta"]=0.1
beta["implicit"]=False
beta["normalize_similarity"]=False

ials={}
ials["epochs"]=45
ials["num_factors"]=50
ials["alpha"]=10.0
ials["epsilon"]=10.0
ials["reg"]=1e-07
ials["init_mean"]=1.0
ials["init_std"]=0.6773548197826565





In [7]:
recs = [
UserKNNCFRecommender,
P3alphaRecommender,
ItemIcmKNNCFRecommender,
UserIcmKNNCFRecommender,
ItemKNNCFRecommender,
RP3betaRecommender
]

fits = [userknn,alpha,iicm,uicm,itemknn,beta]
inits = [only_URM,only_URM,also_ICM,also_ICM,only_URM,only_URM]

#fits = [userknn,alpha,iicm,uicm]#,ials, beta,cy,itemknn]
#inits = [only_URM,only_URM,also_ICM,also_ICM]#,only_URM,only_URM,cython,only_URM]

In [8]:
w = optimize_weights(URM_all,10,recs,inits,fits,levels=4,weights = None, index = 0)


optimize 1
WeightedHybridScoreRecommender: URM Detected 37 (0.47 %) cold users.
WeightedHybridScoreRecommender: URM Detected 1987 (7.65 %) cold items.
UserKNNCFRecommender: URM Detected 37 (0.47 %) cold users.
UserKNNCFRecommender: URM Detected 1987 (7.65 %) cold items.
P3alphaRecommender: URM Detected 37 (0.47 %) cold users.
P3alphaRecommender: URM Detected 1987 (7.65 %) cold items.
VALIDATION 1 with trials [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
weights : [1, 0.0001]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 6673.11 column/sec, elapsed time 0.02 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
------FITTING END------
EvaluatorHoldout: Processed 4962 ( 100.00% ) in 14.95 sec. Users per second: 332
MAP 0.06094284571054443 WITH WEIGHTS [1, 0.0001]
weights : [1, 0.001]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 7555.23 column/

MAP 0.057451780713033515 WITH WEIGHTS [1, 0.01]
weights : [1, 0.1]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 8146.22 column/sec, elapsed time 0.02 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
------FITTING END------
EvaluatorHoldout: Processed 4949 ( 100.00% ) in 15.10 sec. Users per second: 328
MAP 0.05863787086457722 WITH WEIGHTS [1, 0.1]
weights : [1, 1]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 7561.56 column/sec, elapsed time 0.02 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
------FITTING END------
EvaluatorHoldout: Processed 4949 ( 100.00% ) in 14.36 sec. Users per second: 345
MAP 0.05968083153125342 WITH WEIGHTS [1, 1]
weights : [1, 10]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 8397.45 column/sec, elapsed time 0.02 min
--

Similarity column 7947 ( 100 % ), 5983.66 column/sec, elapsed time 0.02 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
------FITTING END------
EvaluatorHoldout: Processed 5024 ( 100.00% ) in 16.58 sec. Users per second: 303
MAP 0.05744255562090059 WITH WEIGHTS [1, 0.0001]
weights : [1, 0.001]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 7747.39 column/sec, elapsed time 0.02 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
------FITTING END------
EvaluatorHoldout: Processed 5024 ( 100.00% ) in 16.54 sec. Users per second: 304
MAP 0.05743215053883947 WITH WEIGHTS [1, 0.001]
weights : [1, 0.01]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 7155.10 column/sec, elapsed time 0.02 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
------FITTING END------
EvaluatorHoldout: Processed 5024 ( 100.00% ) in 16.46 sec. User

------FITTING END------
EvaluatorHoldout: Processed 5005 ( 100.00% ) in 24.51 sec. Users per second: 204
MAP 0.06464034167520552 WITH WEIGHTS [1, 1]
weights : [1, 10]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 7399.49 column/sec, elapsed time 0.02 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
------FITTING END------
EvaluatorHoldout: Processed 5005 ( 100.00% ) in 16.61 sec. Users per second: 301
MAP 0.0638293207170757 WITH WEIGHTS [1, 10]
weights : [1, 100]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 7959.46 column/sec, elapsed time 0.02 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
------FITTING END------
EvaluatorHoldout: Processed 5005 ( 100.00% ) in 14.70 sec. Users per second: 340
MAP 0.06385152640254672 WITH WEIGHTS [1, 100]
WeightedHybridScoreRecommender: URM Detected 41 (0.52 %) cold users.
WeightedHyb

Similarity column 7947 ( 100 % ), 7674.30 column/sec, elapsed time 0.02 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
------FITTING END------
EvaluatorHoldout: Processed 5008 ( 100.00% ) in 16.41 sec. Users per second: 305
MAP 0.060102416097710415 WITH WEIGHTS [1, 0.19999999999999996]
weights : [1, 0.5]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 7636.85 column/sec, elapsed time 0.02 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
------FITTING END------
EvaluatorHoldout: Processed 5008 ( 100.00% ) in 16.47 sec. Users per second: 304
MAP 0.06082058734998541 WITH WEIGHTS [1, 0.5]
weights : [1, 0.8]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 8753.34 column/sec, elapsed time 0.02 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
------FITTING END------
EvaluatorHoldout: Processed 5008 ( 100.00% ) in 17.80 

...

MAP 0.05921881129826285 WITH WEIGHTS [1, 0.736]
weights : [1, 0.7507199999999999]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 10281.43 column/sec, elapsed time 0.01 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
------FITTING END------
EvaluatorHoldout: Processed 4976 ( 100.00% ) in 13.81 sec. Users per second: 360
MAP 0.0591369992884464 WITH WEIGHTS [1, 0.7507199999999999]
weights : [1, 0.7727999999999999]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 8091.17 column/sec, elapsed time 0.02 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
------FITTING END------
EvaluatorHoldout: Processed 4976 ( 100.00% ) in 21.41 sec. Users per second: 232
MAP 0.059139587771365465 WITH WEIGHTS [1, 0.7727999999999999]
weights : [1, 0.79488]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similari

Similarity column 7947 ( 100 % ), 11097.49 column/sec, elapsed time 0.01 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
------FITTING END------
EvaluatorHoldout: Processed 4994 ( 100.00% ) in 10.39 sec. Users per second: 481
MAP 0.06390823094541356 WITH WEIGHTS [1, 0.730112]
weights : [1, 0.73232]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 10840.31 column/sec, elapsed time 0.01 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
------FITTING END------
EvaluatorHoldout: Processed 4994 ( 100.00% ) in 10.35 sec. Users per second: 482
MAP 0.06396396449233654 WITH WEIGHTS [1, 0.73232]
weights : [1, 0.734528]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 10162.01 column/sec, elapsed time 0.01 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
------FITTING END------
EvaluatorHoldout: Processed 4994 ( 100.00% ) in 10

MAP 0.06208593157907829 WITH WEIGHTS [1, 0.736]
weights : [1, 0.7507199999999999]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 9344.25 column/sec, elapsed time 0.01 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
------FITTING END------
EvaluatorHoldout: Processed 4989 ( 100.00% ) in 11.68 sec. Users per second: 427
MAP 0.06206530595579349 WITH WEIGHTS [1, 0.7507199999999999]
weights : [1, 0.7727999999999999]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 11510.57 column/sec, elapsed time 0.01 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
------FITTING END------
EvaluatorHoldout: Processed 4989 ( 100.00% ) in 11.62 sec. Users per second: 429
MAP 0.06205398437408846 WITH WEIGHTS [1, 0.7727999999999999]
weights : [1, 0.79488]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similari

Similarity column 7947 ( 100 % ), 11794.90 column/sec, elapsed time 0.01 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
--------FITTING IN PROGRESS: ItemKNNCFRecommender-------
Similarity column 25975 ( 100 % ), 4163.38 column/sec, elapsed time 0.10 min
Similarity column 25975 ( 100 % ), 2369.09 column/sec, elapsed time 0.18 min
------FITTING END------
EvaluatorHoldout: Processed 5037 ( 100.00% ) in 13.94 sec. Users per second: 361
MAP 0.06336741199652673 WITH WEIGHTS [1, 0.79488, 0.001]
weights : [1, 0.79488, 0.01]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 11896.95 column/sec, elapsed time 0.01 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
--------FITTING IN PROGRESS: ItemKNNCFRecommender-------
Similarity column 25975 ( 100 % ), 4436.52 column/sec, elapsed time 0.10 min
Similarity column 25975 ( 100 % ), 2418.94 column/sec, elapsed time 0.18 min
------FITTING END------
EvaluatorHold

KeyboardInterrupt: 

In [10]:

cy = {}
cy["epochs"] = 400
cy["topK"] = 900
cy["positive_threshold_BPR"] = 7e-18
cy["learning_rate"] = 0.0002
cy["batch_size"] = 10
cy["sgd_mode"] = "sdg"

cython = {
    "URM_train" : URM_train,
    "recompile_cython" : False,
    "verbose" : False
}


recs = [
UserKNNCFRecommender,
P3alphaRecommender,
ItemIcmKNNCFRecommender,
UserIcmKNNCFRecommender,
ItemKNNCFRecommender,
RP3betaRecommender,
SLIM_BPR_Cython
]
weights = [1,0.79488,1,1,1,1,1]

fits = [userknn,alpha,iicm,uicm,itemknn,beta,cy]
inits = [only_URM,only_URM,also_ICM,also_ICM,only_URM,only_URM,cython]

In [ ]:
w = optimize_weights(URM_all,7,recs,inits,fits,levels=3,weights = weights, index = 2)


optimize 2
WeightedHybridScoreRecommender: URM Detected 38 (0.48 %) cold users.
WeightedHybridScoreRecommender: URM Detected 2015 (7.76 %) cold items.
UserKNNCFRecommender: URM Detected 38 (0.48 %) cold users.
UserKNNCFRecommender: URM Detected 2015 (7.76 %) cold items.
P3alphaRecommender: URM Detected 38 (0.48 %) cold users.
P3alphaRecommender: URM Detected 2015 (7.76 %) cold items.
ItemKNNCFRecommender: URM Detected 38 (0.48 %) cold users.
ItemKNNCFRecommender: URM Detected 2015 (7.76 %) cold items.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
VALIDATION 1 with trials [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
weights : [1, 0.79488, 0.0001]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 10673.92 column/sec, elapsed time 0.01 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
--------FITTING IN PROGRESS: ItemKNNCFRecommender-------
Simila

------FITTING END------
EvaluatorHoldout: Processed 5053 ( 100.00% ) in 14.04 sec. Users per second: 360
MAP 0.062111323890918446 WITH WEIGHTS [1, 0.79488, 0.1]
weights : [1, 0.79488, 1]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 8895.39 column/sec, elapsed time 0.01 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
--------FITTING IN PROGRESS: ItemKNNCFRecommender-------
Similarity column 25975 ( 100 % ), 4314.65 column/sec, elapsed time 0.10 min
Similarity column 25975 ( 100 % ), 2472.51 column/sec, elapsed time 0.18 min
------FITTING END------
EvaluatorHoldout: Processed 5053 ( 100.00% ) in 13.72 sec. Users per second: 368
MAP 0.0664779815126202 WITH WEIGHTS [1, 0.79488, 1]
weights : [1, 0.79488, 10]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 12024.02 column/sec, elapsed time 0.01 min
--------FITTING IN PROGRESS: P3alphaRe

Similarity column 25975 ( 100 % ), 4162.03 column/sec, elapsed time 0.10 min
Similarity column 25975 ( 100 % ), 2232.59 column/sec, elapsed time 0.19 min
------FITTING END------
EvaluatorHoldout: Processed 4939 ( 100.00% ) in 15.92 sec. Users per second: 310
MAP 0.06221229368030444 WITH WEIGHTS [1, 0.79488, 0.0001]
weights : [1, 0.79488, 0.001]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 11211.80 column/sec, elapsed time 0.01 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
--------FITTING IN PROGRESS: ItemKNNCFRecommender-------
Similarity column 25975 ( 100 % ), 4072.67 column/sec, elapsed time 0.11 min
Similarity column 25975 ( 100 % ), 2203.41 column/sec, elapsed time 0.20 min
------FITTING END------
EvaluatorHoldout: Processed 4939 ( 100.00% ) in 15.36 sec. Users per second: 322
MAP 0.06219809264995642 WITH WEIGHTS [1, 0.79488, 0.001]
weights : [1, 0.79488, 0.01]
--------FITTING START-------
----

Similarity column 25975 ( 100 % ), 2325.48 column/sec, elapsed time 0.19 min
Similarity column 25975 ( 100 % ), 2195.17 column/sec, elapsed time 0.20 min
------FITTING END------
EvaluatorHoldout: Processed 4993 ( 100.00% ) in 14.66 sec. Users per second: 341
MAP 0.04540900722841147 WITH WEIGHTS [1, 0.79488, 10]
weights : [1, 0.79488, 100]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 11717.07 column/sec, elapsed time 0.01 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
--------FITTING IN PROGRESS: ItemKNNCFRecommender-------
Similarity column 25975 ( 100 % ), 4153.74 column/sec, elapsed time 0.10 min
Similarity column 25975 ( 100 % ), 2320.05 column/sec, elapsed time 0.19 min
------FITTING END------
EvaluatorHoldout: Processed 4993 ( 100.00% ) in 13.26 sec. Users per second: 377
MAP 0.025718225375692263 WITH WEIGHTS [1, 0.79488, 100]
WeightedHybridScoreRecommender: URM Detected 42 (0.53 %) cold users.


Similarity column 7947 ( 100 % ), 12222.51 column/sec, elapsed time 0.01 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
--------FITTING IN PROGRESS: ItemKNNCFRecommender-------
Similarity column 25975 ( 100 % ), 4152.07 column/sec, elapsed time 0.10 min
Similarity column 25975 ( 100 % ), 2454.96 column/sec, elapsed time 0.18 min
------FITTING END------
EvaluatorHoldout: Processed 4991 ( 100.00% ) in 18.12 sec. Users per second: 275
MAP 0.06496412937341128 WITH WEIGHTS [1, 0.79488, 0.01]
weights : [1, 0.79488, 0.1]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 4538.17 column/sec, elapsed time 0.03 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
--------FITTING IN PROGRESS: ItemKNNCFRecommender-------
Similarity column 25975 ( 100 % ), 3808.79 column/sec, elapsed time 0.11 min
Similarity column 25975 ( 100 % ), 1120.92 column/sec, elapsed time 0.39 min
------FITTING END------
EvaluatorHoldout

...

Similarity column 7947 ( 100 % ), 8773.60 column/sec, elapsed time 0.02 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
--------FITTING IN PROGRESS: ItemKNNCFRecommender-------
Similarity column 25975 ( 100 % ), 3906.33 column/sec, elapsed time 0.11 min
Similarity column 25975 ( 100 % ), 2102.25 column/sec, elapsed time 0.21 min
------FITTING END------
EvaluatorHoldout: Processed 5040 ( 100.00% ) in 16.30 sec. Users per second: 309
MAP 0.06431129138346979 WITH WEIGHTS [1, 0.79488, 0.19999999999999996]
weights : [1, 0.79488, 0.5]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 9021.26 column/sec, elapsed time 0.01 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
--------FITTING IN PROGRESS: ItemKNNCFRecommender-------
Similarity column 25975 ( 100 % ), 2485.72 column/sec, elapsed time 0.17 min
Similarity column 25975 ( 100 % ), 1409.81 column/sec, elapsed time 0.31 min
------FITTING END------
Ev

--------FITTING IN PROGRESS: ItemKNNCFRecommender-------
Similarity column 25975 ( 100 % ), 1117.12 column/sec, elapsed time 0.39 min
Similarity column 25975 ( 100 % ), 703.78 column/sec, elapsed time 0.62 min
------FITTING END------
EvaluatorHoldout: Processed 3000 ( 60.07% ) in 37.28 sec. Users per second: 80
EvaluatorHoldout: Processed 4994 ( 100.00% ) in 59.91 sec. Users per second: 83
MAP 0.06864989067127461 WITH WEIGHTS [1, 0.79488, 1]
weights : [1, 0.79488, 3.0]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 4758.57 column/sec, elapsed time 0.03 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
P3alphaRecommender: Processed 21200 ( 81.62% ) in 1.00 minutes. Rows per second: 353
--------FITTING IN PROGRESS: ItemKNNCFRecommender-------
Similarity column 25975 ( 100 % ), 937.59 column/sec, elapsed time 0.46 min
Similarity column 25975 ( 100 % ), 625.56 column/sec, elapsed time 0.69 min
------FITTING E

EvaluatorHoldout: Processed 4986 ( 100.00% ) in 1.23 min. Users per second: 68
MAP 0.055149166027183893 WITH WEIGHTS [1, 0.79488, 6.0]
weights : [1, 0.79488, 9.0]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 1414.99 column/sec, elapsed time 0.09 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
P3alphaRecommender: Processed 10000 ( 38.50% ) in 1.01 minutes. Rows per second: 166
P3alphaRecommender: Processed 20000 ( 77.00% ) in 2.02 minutes. Rows per second: 165
--------FITTING IN PROGRESS: ItemKNNCFRecommender-------
Similarity column 23000 ( 89 % ), 766.56 column/sec, elapsed time 0.50 min
Similarity column 25975 ( 100 % ), 772.14 column/sec, elapsed time 0.56 min
Similarity column 18374 ( 71 % ), 603.23 column/sec, elapsed time 0.51 min
Similarity column 25975 ( 100 % ), 646.66 column/sec, elapsed time 0.67 min
------FITTING END------
EvaluatorHoldout: Processed 4000 ( 80.22% ) in 34.83 sec. Users per

EvaluatorHoldout: Processed 5030 ( 100.00% ) in 1.09 min. Users per second: 77
MAP 0.06189263111179222 WITH WEIGHTS [1, 0.79488, 4.5]
weights : [1, 0.79488, 5.4]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 2284.43 column/sec, elapsed time 0.06 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
P3alphaRecommender: Processed 23000 ( 88.55% ) in 1.01 minutes. Rows per second: 380
--------FITTING IN PROGRESS: ItemKNNCFRecommender-------
Similarity column 25975 ( 100 % ), 1076.02 column/sec, elapsed time 0.40 min
Similarity column 25975 ( 100 % ), 888.31 column/sec, elapsed time 0.49 min
------FITTING END------
EvaluatorHoldout: Processed 3000 ( 59.64% ) in 31.74 sec. Users per second: 95
EvaluatorHoldout: Processed 5030 ( 100.00% ) in 48.06 sec. Users per second: 105
MAP 0.05782894178767145 WITH WEIGHTS [1, 0.79488, 5.4]
WeightedHybridScoreRecommender: URM Detected 46 (0.58 %) cold users.
WeightedHybridScor

MAP 0.07051304238259588 WITH WEIGHTS [1, 0.79488, 2.76]
weights : [1, 0.79488, 2.85]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 7893.77 column/sec, elapsed time 0.02 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
--------FITTING IN PROGRESS: ItemKNNCFRecommender-------
Similarity column 25975 ( 100 % ), 2539.02 column/sec, elapsed time 0.17 min
Similarity column 25975 ( 100 % ), 1393.28 column/sec, elapsed time 0.31 min
------FITTING END------
EvaluatorHoldout: Processed 5011 ( 100.00% ) in 25.62 sec. Users per second: 196
MAP 0.07004626411715333 WITH WEIGHTS [1, 0.79488, 2.85]
weights : [1, 0.79488, 2.94]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 5231.60 column/sec, elapsed time 0.03 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
--------FITTING IN PROGRESS: ItemKNNCFRecommender-------
Similarity column 25975

In [12]:
weights

[1, 0.79488, 2.76, 0.14400000000000002, 2.76, 1, 1]

In [13]:
recs = [
UserKNNCFRecommender,
P3alphaRecommender,
ItemIcmKNNCFRecommender,
UserIcmKNNCFRecommender,
ItemKNNCFRecommender,
RP3betaRecommender,
SLIM_BPR_Cython
]
weights = [1,0.79488, 2.76,0.144,1,1,1]

fits = [userknn,alpha,iicm,uicm,itemknn,beta,cy]
inits = [only_URM,only_URM,also_ICM,also_ICM,only_URM,only_URM,cython]

In [ ]:
w = optimize_weights(URM_all,7,recs,inits,fits,levels=3,weights = weights, index = 4)


optimize 4
WeightedHybridScoreRecommender: URM Detected 33 (0.42 %) cold users.
WeightedHybridScoreRecommender: URM Detected 2012 (7.75 %) cold items.
UserKNNCFRecommender: URM Detected 33 (0.42 %) cold users.
UserKNNCFRecommender: URM Detected 2012 (7.75 %) cold items.
P3alphaRecommender: URM Detected 33 (0.42 %) cold users.
P3alphaRecommender: URM Detected 2012 (7.75 %) cold items.
ItemKNNCFRecommender: URM Detected 33 (0.42 %) cold users.
ItemKNNCFRecommender: URM Detected 2012 (7.75 %) cold items.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
UserKNNCFRecommender: URM Detected 33 (0.42 %) cold users.
UserKNNCFRecommender: URM Detected 2012 (7.75 %) cold items.
UserKNNCFRecommender: URM Detected 33 (0.42 %) cold users.
UserKNNCFRecommender: URM Detected 28 (0.14 %) cold items.
ItemKNNCFRecommender: URM Detected 33 (0.42 %) cold users.
ItemKNNCFRecommender: URM Detected 2012 (7.75 %) cold items.
VALIDATION 1 with 

UserKNNCFRecommender: URM Detected 28 (0.14 %) cold items.
ItemKNNCFRecommender: URM Detected 40 (0.50 %) cold users.
ItemKNNCFRecommender: URM Detected 2015 (7.76 %) cold items.
VALIDATION 2 with trials [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
weights : [1, 0.79488, 2.76, 0.144, 0.0001]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 11737.78 column/sec, elapsed time 0.01 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
--------FITTING IN PROGRESS: ItemKNNCFRecommender-------
Similarity column 25975 ( 100 % ), 4818.02 column/sec, elapsed time 0.09 min
Similarity column 25975 ( 100 % ), 2677.45 column/sec, elapsed time 0.16 min
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 13792.18 column/sec, elapsed time 0.01 min
Similarity column 7947 ( 100 % ), 2207.93 column/sec, elapsed time 0.06 min
--------FITTING IN PROGRESS: ItemKNNCFRecommender-------
Similarity column

Similarity column 7947 ( 100 % ), 12774.74 column/sec, elapsed time 0.01 min
--------FITTING IN PROGRESS: P3alphaRecommender-------
--------FITTING IN PROGRESS: ItemKNNCFRecommender-------
Similarity column 25975 ( 100 % ), 4849.78 column/sec, elapsed time 0.09 min
Similarity column 25975 ( 100 % ), 2632.78 column/sec, elapsed time 0.16 min
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similarity column 7947 ( 100 % ), 13804.24 column/sec, elapsed time 0.01 min
Similarity column 7947 ( 100 % ), 2211.86 column/sec, elapsed time 0.06 min
--------FITTING IN PROGRESS: ItemKNNCFRecommender-------
Similarity column 25975 ( 100 % ), 4159.34 column/sec, elapsed time 0.10 min
------FITTING END------
EvaluatorHoldout: Processed 5021 ( 100.00% ) in 18.34 sec. Users per second: 274
MAP 0.06625850324454868 WITH WEIGHTS [1, 0.79488, 2.76, 0.144, 0.0001]
weights : [1, 0.79488, 2.76, 0.144, 0.001]
--------FITTING START-------
--------FITTING IN PROGRESS: UserKNNCFRecommender-------
Similari

# FINAL FIT

# CHANGE URM TO FITS

In [ ]:


recommender = WeightedHybridScoreRecommender(URM_all, recs, inits)

weights = [1,w1,w2,w3]
recommender.fit(fits,weights)



In [ ]:
import time
URM_file = open(r"../datasets/data_target_users_test.csv", 'r')

URM_file.seek(0)
URM_tuples = []
URM_file.readline()#questa è importante di cristo c'è riga di introduzione
for line in URM_file:
    URM_tuples.append(int(line.replace("\n","")))

usersToPred = np.array(URM_tuples)

start_time = time.time()#BUONE METRICHE PER IL TEMPO QUI
import csv

with open('WWWWWoutputProva.csv', mode='w',newline='') as recFile:
    
    recFile = csv.writer(recFile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    recFile.writerow(['user_id', 'item_list'])
    recs=recommender.recommend(usersToPred, cutoff=10)
    for user_id,rec in zip(usersToPred,recs):
        rec_string =' '.join(map(str, rec[:10]))
        recFile.writerow([user_id, rec_string])
    
end_time = time.time()
print("Ha finito in questo tempo: "+str(end_time-start_time)+" secondi")

# OPTIMIZE WITH OPTIMIZER

In [ ]:
only_URM = {
    "URM_train" : URM_train
}

cython = {
    "URM_train" : URM_train,
    "recompile_cython" : False,
    "verbose" : False
also_ICM = {
    "URM_train" : URM_train,
    "ICM" : ICM_all
}

recs = [
    P3alphaRecommender,
    UserIcmKNNCFRecommender,
    ItemKNNCFRecommender
#     UserKNNCFRecommender,
#     ItemIcmKNNCFRecommender,
#     SLIM_BPR_Cython
]

inits = [
    only_URM,also_ICM,only_URM#,only_URM,also_ICM, cython
]



fits = [alpha,userknn,uicm]#,iicm,itemknn,cy]

In [ ]:
from optimize_weights import optimize_weights

In [ ]:
w = optimize_weights(URM_train,evaluator_validation,recs, inits, fits)

In [ ]:
w

In [ ]:
recs = [
    P3alphaRecommender,
    UserIcmKNNCFRecommender,
    ItemKNNCFRecommender
]

inits = [
    only_URM,also_ICM,only_URM
]



fits = [alpha,uicm,itemknn]

In [ ]:
recommender = WeightedHybridScoreRecommender(URM_all, recs, inits)

weights = w
recommender.fit(fits,weights)

In [ ]:
import time
URM_file = open(r"../datasets/data_target_users_test.csv", 'r')

URM_file.seek(0)
URM_tuples = []
URM_file.readline()#questa è importante di cristo c'è riga di introduzione
for line in URM_file:
    URM_tuples.append(int(line.replace("\n","")))

usersToPred = np.array(URM_tuples)

start_time = time.time()#BUONE METRICHE PER IL TEMPO QUI
import csv

with open('WWWoutputProva.csv', mode='w',newline='') as recFile:
    
    recFile = csv.writer(recFile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    recFile.writerow(['user_id', 'item_list'])
    recs=recommender.recommend(usersToPred, cutoff=10)
    for user_id,rec in zip(usersToPred,recs):
        rec_string =' '.join(map(str, rec[:10]))
        recFile.writerow([user_id, rec_string])
    
end_time = time.time()
print("Ha finito in questo tempo: "+str(end_time-start_time)+" secondi")